# Change detection framework (CDF)

This script is used for training and inference an anomaly detection in time series of EDFAs pump current.

## Import necessary modules

We will use pandas and numpy for data handling, scikit-learn for scaling and the buildin PCA and the custom clustering algorithms.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from entropyAnlysis import EA
from fuzzyCMeans import FCM
from probabilisticClustering import ProbC
from possibilisticClustering import PossC

## Makros

We will need to define certain thresholds and parameters for the algorithms and the stop criteriums

In [ ]:
THRESHOLD_ENTROPY = 2
THRESHOLD_PCA = 0.99

TRAINLOOP_CLUSTER = 2
TRAINLOOP_EPS = 1E-4
TRAINLOOP_MAXEPOCHS = 1

ALGOHRITHM_BETA = 2
ALGOHRITHM_ETA = 0.001

## Training
### Loading the data

In [ ]:
df = pd.read_excel("data/X_train.xlsx", header=0)
data = np.array(df)

### Scaling

In [ ]:
data = StandardScaler().fit_transform(data)

### Entropy analysis

In [ ]:
ea = EA(THRESHOLD_ENTROPY)

df_reduced = ea.run(data, df)

### PCA

In [ ]:
pca_data = PCA(THRESHOLD_PCA)

principalComponents = pca_data.fit_transform(np.array(df_reduced))

### Shuffle and scale

In [ ]:
np.random.shuffle(principalComponents)

principalComponents = StandardScaler().fit_transform(
    principalComponents
)

X = principalComponents

### Train the Fuzzy C-Means algorithm

In [ ]:
fcm = FCM(
    data_shape=X.shape,
    trainloop_cluster=TRAINLOOP_CLUSTER,
    trainloop_maxEpochs=TRAINLOOP_MAXEPOCHS,
    trainloop_eps=TRAINLOOP_EPS
)

fcm.train(X)

### Train the probabilistic clustering algorithm

In [ ]:
probC = ProbC(
    data_shape=X.shape,
    trainloop_cluster=TRAINLOOP_CLUSTER,
    trainloop_maxEpochs=TRAINLOOP_MAXEPOCHS,
    trainloop_eps=TRAINLOOP_EPS,
    algorithm_beta=ALGOHRITHM_BETA,
    algorithm_eta=ALGOHRITHM_ETA
)

probC.train(X)

### Train the possibilistic clustering algorithm

In [ ]:
possC = PossC(
    data_shape=X.shape,
    trainloop_cluster=TRAINLOOP_CLUSTER,
    trainloop_maxEpochs=TRAINLOOP_MAXEPOCHS,
    trainloop_eps=TRAINLOOP_EPS,
    algorithm_beta=ALGOHRITHM_BETA,
    algorithm_eta=ALGOHRITHM_ETA
)

possC.train(X)

## Inference


In [ ]:
df_test = pd.read_excel("data/X_test.xlsx", header=0)

data_test = np.array(df_test)

df_test_reduced = ea.invoke(df_test)

principalComponents_test = pca_data.transform(np.array(df_test_reduced))

principalComponents_test = StandardScaler().fit_transform(principalComponents_test)

X_test = principalComponents_test

fcm_test = fcm.invoke(X_test)

probC_test = probC.invoke(X_test)

possC_test = possC.invoke(X_test)